In [1]:
#from https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding 
from keras.preprocessing import sequence
import keras
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
import tensorflow as tf

Using TensorFlow backend.


In [2]:
#reproducibility
from keras import backend as K
import os
import random as rn
os.environ['PYTHONHASHSEED'] ='0'
#Below is necessary for starting Numpy generated 
#random numbers in a well-defined initial state
np.random.seed(42)

#Below is necessary for starting core Python generated 
#random numbers in a well-defined initial state
rn.seed(12345)
'''force tensorflow to use single thread
   multiple threads are the potential source
   of non-reproducible results
'''
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
#The below tf.set_random_seed() will make random number generation in
#Tensorflow backend have well defined initial state
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [3]:
#Define activation functions
def tf_sqnlsig(x):
    u=tf.clip_by_value(x,-2,2)
    a = u
    b= tf.negative(tf.abs(u))
    wsq = (tf.multiply(a,b))/4.0
    y = tf.add(tf.multiply(tf.add(u,wsq),0.5),0.5)
    return y
get_custom_objects().update({'custom_activation': Activation(tf_sqnlsig)})
def tf_sqnl(x): 
    u=tf.clip_by_value(x,-2,2)
    a = u
    b= tf.negative(tf.abs(u))
    wsq = (tf.multiply(a,b))/4.0
    y = tf.add(u,wsq)
    return y
get_custom_objects().update({'custom_activation': Activation(tf_sqnl)})

def tf_tansig(x): 
    w=tf.multiply(tf.negative(2.0), x)
    u=tf.exp(w)
    a = 1.0 + u
    b= 2.0/a
    y = b - 1.0
    return y
get_custom_objects().update({'custom_activation': Activation(tf_tansig)})

def tf_logsig(x): 
    u=tf.exp(tf.negative(x)) 
    a = tf.add(u,1.0)
    y= 1.0/a
    return y
get_custom_objects().update({'custom_activation': Activation(tf_logsig)})

In [5]:
#load data
top_words = 5000 #only keep the top 5000 words, zero for the rest
(x_train, y_train), (x_test,y_test) = imdb.load_data(num_words=top_words)

In [6]:
#truncate and pad input sequence to perform computation in Keras
max_review_length = 500
x_train = sequence.pad_sequences(x_train,maxlen=max_review_length)
x_test = sequence.pad_sequences(x_test,maxlen=max_review_length)

In [14]:
# create the model
#Embedding : first layer uses 32 length vectors to represent each word
embedding_vecor_length = 32
model = Sequential()
#input layer
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
#model.add(Dropout(0.5))
#LSTM layer
#keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid'
#model.add(LSTM(100)) #number of neurons = 100
#model.add(keras.layers.LSTM(100 , activation='tanh', recurrent_activation='sigmoid'))
model.add(keras.layers.LSTM(100 , activation=tf_sqnl, recurrent_activation=tf_sqnlsig))
#model.add(keras.layers.LSTM(100 , activation=tf_tansig, recurrent_activation=tf_logsig))
#model.add(Dropout(0.5))
#model.add(Dense(1, activation='sigmoid'))
#output layer
#model.add(Dense(1, activation='sigmoid'))
#model.add(Dense(1, activation=tf_logsig))
model.add(Dense(1, activation=tf_sqnlsig))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 604s 24ms/step - loss: 0.4882 - acc: 0.7616 - val_loss: 0.4121 - val_acc: 0.8265
Epoch 2/2
25000/25000 [==============================] - 616s 25ms/step - loss: 0.2976 - acc: 0.8798 - val_loss: 0.3199 - val_acc: 0.8689


In [ ]:
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))